# Tutorial 2
## tutorial 2 content
1. AutoGrad是什么（救命是新名词）
2. 优化器怎么用
3. 使用pytorch的三种level(
  高中低)

## AutoGrad是什么
好的，这里我们来介绍一个新东西，autograd（自动求导）。这是一个很重要的概念，也是使用pytorch这种深度
学习框架的一个最大优势。下边的东西可能需要大家有些深度学习的基础才能看懂，我尽力给大家讲的清楚点。

我先大致的描述一下深度学习的蓝图。

对于优化问题（别问我什么是优化问题，先记住这是个优化问题），我们希望使用深度学习完成一个任务，我举个例子来讲，比如我们希望用深度学习来判断一张图片是不是奶龙，那么我们需要怎么做呢？把深度学习想象成一个黑盒子，你把奶龙的照片输进去，然后黑盒子读取这个照片，输出0或1（0代表不是奶龙，1代表是奶龙）。（实际问题我们并不是直接输出0或1，而是输出是0的概率或1的概率）很好，这就是深度学习。但是我们可以想象得到，这个黑盒子刚被创建的时候，是没有识别奶龙的能力的，我们需要训练他识别奶龙！既然要训练，我们总需要一个指标来衡量这个黑盒子识别奶龙的能力吧，这种时候我们引入一个损失函数loss，这个函数用来描述黑盒子识别奶龙的能力，它识别能力越强，loss越小，识别能力越弱，loss越大，所以我们的任务就是把loss变小！好了，这就是优化问题名字的由来，我们把训练任务抽象成了一个数学问题，优化这个损失函数（让它变小）

ok，我们继续，接下来，我们要拆开黑盒子，看看里边是什么东西。我们可以这么理解，这个黑盒子有很多参数，用来计算这个照片是奶龙的概率，也就是说，这个黑盒子会给照片打分，越像奶龙，分数越高，越不像，分数越低。然后最后我们会给出我们的0，1输出，黑盒子会对于这张照片是不是奶龙给出自己的答案。

但是！人总会犯错嘛，这个黑盒子刚被你创建出来的时候，就像刚出生一样，啥也不懂，哪知道什么是奶龙呢？它一开始给的答案肯定是随机的，这个时候，我们就根据黑盒子判断的好不好，用一个损失函数公式，来衡量它的识别奶龙的能力。然后我们就来到了最重要的问题！怎么减小损失函数。

注意到，黑盒子用一堆参数得到了对奶龙的识别结果，我们据此得到了损失函数。我们不妨灵光一现，损失函数，不就是黑盒子里各个参数的函数吗！我们可以写$L=L(x1, x2, ……, xn)$里边x是对应的参数，如果我们算出来L关于各个参数的梯度，然后让这些参数往梯度下降的地方走那么一丢丢，是不是L就减小了！这就是梯度下降的由来，更准确更权威的讲解大家还是要去看看论文噢。

好的，我们找到了优化损失函数的想法，但是算这个梯度，好麻烦呀，要求好多导数……等等！autograd是不是可以自动求导来着？没错，autograd就是pytorch这个框架的魅力，他让你无需手搓loss关于每个参数的梯度，它会直接帮你算好。
在大模型中，参数量往往是上亿的，如果没有这种autograd功能，我们就要求导上亿次，有了autograd，这一切就可以被大大简化！

下边展示一个autograd的例子。

In [1]:
import torch

# 创建一个张量，并设置 requires_grad=True 以跟踪其计算历史
x = torch.tensor(2.0, requires_grad=True)

# 定义一个函数 y = x^2 + 3x + 1
y = x**2 + 3*x + 1

# 使用 autograd 计算 y 关于 x 的梯度
y.backward()

# 打印梯度 dy/dx
print("x:", x)
print("y:", y)
print("Gradient of y with respect to x:", x.grad)

x: tensor(2., requires_grad=True)
y: tensor(11., grad_fn=<AddBackward0>)
Gradient of y with respect to x: tensor(7.)


## 优化器怎么用
我们刚刚提到了梯度下降的想法，也就是可以创建一个类，专门用来处理优化函数的问题。但其实优化问题不仅仅有最简单的SGD，还有很多更好的变种，比如SGDmomentum，Adam等等（别的我也忘了）。所以我们在实际问题中要选择更合适的优化器来对损失函数做优化。然后注意，损失函数也有很多很多的形式，我们也需要选择合适的损失函数。下边是个小栗子。

In [ ]:
import torch.optim as optim
import torch.nn as nn

class model(nn.Module):
    pass #这里一般用来写我们的具体模型的架构，这里先略去
'''
我们先假设这个模型有个输出，输出y_pred，来代表它识别这张图片是不是奶龙，y_pred可以是0到1的一个数字，越靠近1代表
是奶龙的概率越大，越靠近0代表越不可能是奶龙（实际中这里一般用logistic输出两个概率，我这里先举个简单的示例
帮助大家理解）然后我们有y，代表这个图片是否真的是奶龙，是的话y就等于1，不是的话y就等于0
'''

criterion = torch.nn.Softmax()
#实际中这里应该不会用softmax，softmax用交叉熵损失函数，这里是个示例
#简单理解，criterion是我们用来算损失函数的一个小工具

loss = criterion(y_pred, y)
#这里计算损失函数了！

optimizer = optim.SGD(model.parameters(), lr=0.01)  # 随机梯度下降优化器，这里输入了model的参数

optimizer.zero_grad()  # 清空梯度
loss.backward()  # 计算梯度

# 更新参数
optimizer.step()

嗯，希望我解释清楚了。

## 使用pytorch的三种level
我在学stanford CS231N的时候，它把pytorch分成了三种level的使用，这个level是针对我们搭建模型时使用的方法。
注意，我们越高级的level，一般编写架构会很简洁以及方便，但是代价是设计的模型的灵活性会下降。
打个比方，我们要搭积木，我们可以那做好的积木拼起来，那样我们会搭出一个房子，但是我们使用的组件
全是别人做好的，我们不能更自由的决定房子的一些细节，比如屋顶样式、支柱样式等等。
但如果我们自己做每个小块的积木，虽然麻烦了一些，但是我们可以完全的掌握房子的每个细节。

下边依次展示pytorch的三种level编写。

最高级别，nn.sequential(有一说一它好像不止sequential这种容器，这里就单拿它做示例)
sequential是一个容器，我们深度学习往往会堆叠很多“层”，理解这个层的概念需要你了解深度学习，这里不展开讲，但我想把它解释为榴莲千层，我们会每层每层这样堆起来，判断是不是奶龙的时候，我们的计算逐层流过。
这里sequential用来帮你直接装好所有的层，其他的一律不用考虑。

In [ ]:
model = nn.Sequential(
    nn.Linear(),
    nn.ReLU(),
    nn.Conv2d()
)
# 这里就是一个已经编好的架构（model）了，我们并没有写很多的代码，只是把那些看起来酷炫的层堆起来

nn.Module，这是一个中级别的用法，我们需要定义前向传播方式，但是可以自由决定更丰富的前向传播方法。

In [ ]:
class model(nn.Module): #这里需要继承nn.Module的父类，有大用！
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs) #这里调用父类的初始化方法，然后我们就可以初始化我们的连接层了
        self.Linear = nn.Linear()
        self.relu = nn.ReLU()
        self.conv2d = nn.Conv2d() #我们初始化了三个很酷炫的“层，并赋给这个model类里”
    def forward(self, x): #x是我们的输入！就是奶龙的图片
        x = self.Linear(x) #把x变成通过全连接层的新数值，再传给下个层
        x = self.relu(x) 
        x = self.conv2d(x)
        return x
#这里也是一个写好的model了，需要我们手动实现init和forward（前向传播）的方法

最低level！我太懒了，放一个AI写的简易版本，给大家解释一下。

In [ ]:
import torch

# 手动定义参数，这里是超参数，大家可以了解一下超参数这个概念
input_size = 10
hidden_size = 50
output_size = 1

# 初始化权重和偏置（救命，所有的参数还需要自己去初始化）
W1 = torch.randn(input_size, hidden_size, requires_grad=True)  # 输入层到隐藏层的权重
b1 = torch.randn(hidden_size, requires_grad=True)             # 隐藏层的偏置
W2 = torch.randn(hidden_size, output_size, requires_grad=True)  # 隐藏层到输出层的权重
b2 = torch.randn(output_size, requires_grad=True)              # 输出层的偏置

# 定义前向传播
def forward(x):
    x = torch.matmul(x, W1) + b1  # 输入层到隐藏层
    x = torch.relu(x)             # 激活函数
    x = torch.matmul(x, W2) + b2  # 隐藏层到输出层
    return x

def loss_fn(y_pred, y_true):
    return ((y_pred - y_true) ** 2).mean()

# 测试数据
x = torch.randn(1, 10)  # 输入数据 (batch_size=1, input_size=10)
y_true = torch.tensor([[1.0]])  # 真实标签

# 训练循环
learning_rate = 0.01
for epoch in range(100):
    # 前向传播
    y_pred = forward(x)
    
    # 计算损失
    loss = loss_fn(y_pred, y_true)
    
    # 反向传播
    loss.backward()
    
    # 手动更新参数（梯度下降）
    with torch.no_grad():  # 禁用梯度计算
        W1 -= learning_rate * W1.grad
        b1 -= learning_rate * b1.grad
        W2 -= learning_rate * W2.grad
        b2 -= learning_rate * b2.grad
        
        # 清空梯度
        W1.grad.zero_()
        b1.grad.zero_()
        W2.grad.zero_()
        b2.grad.zero_()
    
    # 打印损失
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/100], Loss: {loss.item():.4f}')

# 打印最终预测值
print("Final Prediction:", y_pred)

如你所见，这个真的好麻烦，反正我一般也不会用到barebones这种level，一般nn.Module这种

嗯，这是我花了一节线代课和半节大物课写的两个tutorial，苯人能力有限，里边应该会有很多不准确不恰当的地方，大家看到了别骂我，指正就好了。然后，这个tutorial并不能教会你直接使用pytorch，写这个tutorial的目的是让大家对pytorch有个更好的直观理解，深入学习需要大家去查官方文档，其它资料等等。
没了，嗯，大家好好上课。